<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1">Load Data</a></span><ul class="toc-item"><li><span><a href="#Train-Test-Split" data-toc-modified-id="Train-Test-Split-1.1">Train-Test Split</a></span></li></ul></li><li><span><a href="#Clean-Data" data-toc-modified-id="Clean-Data-2">Clean Data</a></span></li><li><span><a href="#Explore" data-toc-modified-id="Explore-3">Explore</a></span></li><li><span><a href="#Simple-Model" data-toc-modified-id="Simple-Model-4">Simple Model</a></span></li><li><span><a href="#Evaluate" data-toc-modified-id="Evaluate-5">Evaluate</a></span><ul class="toc-item"><li><span><a href="#Using-Just-Training-Data" data-toc-modified-id="Using-Just-Training-Data-5.1">Using Just Training Data</a></span></li><li><span><a href="#Validation-(w/-Cross-Validation)" data-toc-modified-id="Validation-(w/-Cross-Validation)-5.2">Validation (w/ Cross-Validation)</a></span></li><li><span><a href="#Evaluate-with-Test" data-toc-modified-id="Evaluate-with-Test-5.3">Evaluate with Test</a></span></li></ul></li></ul></div>

In [15]:
from typing import Optional
import requests
from zipfile import ZipFile
from io import BytesIO

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, confusion_matrix,
)

# Load Data

In [2]:
def load_data(
    url: str,
    filename: str,
    zip_filename: Optional[str] = None,
    **kwargs,
 ) -> pd.DataFrame:
    '''Download zip file with CSV from URL and returns a DataFrame.
    '''
    if zip_filename is None:
        zip_filename = url.split('/')[-1]

    r = requests.get(url)
    files = ZipFile(BytesIO(r.content))
    # No header
    return pd.read_csv(files.open(filename), **kwargs)

def load_spam_data() -> pd.DataFrame:
    '''Returns DataFrame for SMS spam detection (https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection)
    '''
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
    filename = 'SMSSpamCollection'
    df = load_data(url, filename, sep='\t', names=['label', 'text'])
    return df

In [3]:
df = load_spam_data()
display(df.head())
display(df['label'].value_counts())

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


ham     4825
spam     747
Name: label, dtype: int64

## Train-Test Split

In [4]:
def get_train_test_data(X, y) -> tuple:
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=27,
    )
    return X_train, X_test, y_train, y_test

In [5]:
X_train, X_test, y_train, y_test = get_train_test_data(
                                        X=df['text'],
                                        y=df['label'],
)


# Check that this looks good (train & test should look "similar")
print(f'Train (n={y_train.shape[0]:,}):')
for label, count in zip(*np.unique(y_train, return_counts=True)):
    print(f'{label}\t{count/y_train.shape[0]:.2%}') # Normalize Counts

print()
print(f'Test (n={y_test.shape[0]:,}):')
for label, count in zip(*np.unique(y_test, return_counts=True)):
    print(f'{label}\t{count/y_test.shape[0]:.2%}') # Normalize Counts

Train (n=4,457):
ham	86.90%
spam	13.10%

Test (n=1,115):
ham	85.38%
spam	14.62%


# Clean Data

> We'll "skip" this and just say it's clean enough but something #TODO

In [6]:
def get_clean_data(data: pd.Series) -> np.ndarray:
    '''Lower text data
    '''
    # TODO: Actually clean the data with some process
    return data.str.lower().values # Turn it into a numpy array

def get_clean_labels(
    y: pd.Series,
    ham: Optional[str] ='ham',
    spam: Optional[str] ='spam',
) -> np.ndarray:
    '''Convert strings of 'ham' and 'spam' to 0 & 1 (respectively)
    '''
    assert (ham is not None) or (spam is not None), 'Define either ham or spam'
    
    if ham:
        y_clean = np.fromiter(
            iter=(0 if y_ == ham else 1 for y_ in y), 
            dtype=int,
        )
    elif spam:
        y_clean = np.fromiter(
            iter=(1 if y_ == spam else 0 for y_ in y), 
            dtype=int,
        )
    return y_clean

In [7]:
X_train_clean = get_clean_data(X_train)
X_test_clean = get_clean_data(X_test)

In [8]:
# Change ham → 0 & spam → 1
y_train = get_clean_labels(y_train, ham='ham')
y_test = get_clean_labels(y_test, ham='ham')

# Explore

> We'll also "skip" the exploring since we're focusing on the model building right now.
> Just another thing #TODO

# Simple Model

A _very simple model_ is checking if "spammy" words are being used and mark it as spam

> NOTE: We could have a more "realistic" simple model by looking at words that _only_ occur in training spam (or conversely words that only occur in training ham)

In [9]:
# 
class SimpleModel(BaseEstimator, ClassifierMixin):
    ''' Very simple model that checks provided `spam_words` to label as "spam".
    Assumes that first class is "ham" and next class is "spam".
    
    Inspired by https://scikit-learn.org/stable/developers/develop.html#rolling-your-own-estimator
    '''
    
    def __init__(self, spam_words=('free')):
        self.spam_words = spam_words

    def fit(self, X, y):
        self.X_ = X
        self.y_ = y
        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)

        # Spam if text contains any of the spam words
        preds = [
            1 if any(word in X for word in self.spam_words)
            else 0
                for X in X_train_clean
        ]
        return preds
    

In [10]:
common_spam_words = (
    'free',
    'won',
    'win', # variation of "win"
    'money',
)

model = SimpleModel(spam_words=common_spam_words)

In [11]:
model.fit(X_train_clean.reshape(-1,1), y_train)
y_pred = model.predict(X_train_clean.reshape(-1,1))

# Evaluate

## Using Just Training Data

In [16]:
print(f'''Evaluation using Training Data:
    {accuracy_score(y_train, y_pred)=:.5f}
    {recall_score(y_train, y_pred)=:.5f}
    {precision_score(y_train, y_pred)=:.5f}
''')

Evaluation using Training Data:
    accuracy_score(y_train, y_pred)=0.88109
    recall_score(y_train, y_pred)=0.44349
    precision_score(y_train, y_pred)=0.55819



In [20]:
# Percent we got correct for each class
confusion_matrix(y_train, y_pred, normalize='true')

array([[0.94706946, 0.05293054],
       [0.55650685, 0.44349315]])

## Validation (w/ Cross-Validation)

## Evaluate with Test